In [117]:
import nltk
from nltk.stem import WordNetLemmatizer

In [118]:
lemmatizer = WordNetLemmatizer()

In [119]:
import json
import pickle

In [120]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout 
from keras.optimizers import SGD
#SGD Stochastic Gradient Descend
import random


In [121]:
import pandas as pd

In [122]:
#initialize list
words = []
classes = []
documents = []

In [123]:
with open('intent.json','r') as f:
    data = json.load(f)
df = pd.DataFrame(data['intents'])
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tag        20 non-null     object
 1   patterns   20 non-null     object
 2   responses  20 non-null     object
dtypes: object(3)
memory usage: 612.0+ bytes


In [124]:
df['responses'][0]

['Hello! How can I help you today?',
 'Hi there! What can I do for you?',
 "Hey! How's it going?",
 'Good morning! How can I assist you?']

In [125]:
df


,tag,patterns,responses
0,greeting,"[Hi, Hello, Hey there, Good morning, How are y...","[Hello! How can I help you today?, Hi there! W..."
1,weather,"[What's the weather like today?, Is it going t...",[The weather today is sunny with a high of 25°...
2,restaurant_recommendations,"[Can you recommend a good restaurant?, I'm loo...",[Sure! How about [Restaurant A]? They have gre...
3,product_information,"[Tell me about your products., What types of p...",[We offer a wide range of products including e...
4,booking_flights,"[I want to book a flight., Can you help me fin...",[Sure! Where would you like to fly to and from...
5,tech_support,"[I'm having trouble with my [device]., How do ...",[I'm sorry to hear that. What seems to be the ...
6,order_tracking,"[Where is my order?, Can you track my package?...",[Please provide your order number so I can che...
7,account_support,"[I forgot my password., How do I change my ema...",[You can reset your password through the passw...
8,appointment_booking,"[How do I schedule an appointment?, Can I book...",[You can schedule an appointment online throug...
9,general_info,"[What can you tell me about your company?, Whe...","[We are a company specializing in [industry].,..."


In [126]:
data_file = open('intent.json').read()
intents = json.loads(data_file)

In [127]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hello', 'Hey there', 'Good morning', 'How are you?'],
   'responses': ['Hello! How can I help you today?',
    'Hi there! What can I do for you?',
    "Hey! How's it going?",
    'Good morning! How can I assist you?']},
  {'tag': 'weather',
   'patterns': ["What's the weather like today?",
    'Is it going to rain tomorrow?',
    "How's the weather in [city]?",
    "Tell me about today's weather forecast.",
    'Will it be sunny this weekend?'],
   'responses': ['The weather today is sunny with a high of 25°C.',
    "Tomorrow's forecast predicts rain in the afternoon.",
    "In [city], it's currently 20°C and partly cloudy.",
    'Today will be sunny with a slight chance of showers.',
    'This weekend looks like it will be sunny and warm.']},
  {'tag': 'restaurant_recommendations',
   'patterns': ['Can you recommend a good restaurant?',
    "I'm looking for Italian food.",
    'Any sushi restaurants nearby?',
    'Where can I fin

In [128]:
from nltk import word_tokenize

In [129]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = word_tokenize(pattern)
        words.extend(w)

        documents.append((w,intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# document has a tupple of a tokenized version of the question which is in pair with the tag of that question
# (['Hi','how','are','you'],'greeting')
documents
        

[(['Hi'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Hey', 'there'], 'greeting'),
 (['Good', 'morning'], 'greeting'),
 (['How', 'are', 'you', '?'], 'greeting'),
 (['What', "'s", 'the', 'weather', 'like', 'today', '?'], 'weather'),
 (['Is', 'it', 'going', 'to', 'rain', 'tomorrow', '?'], 'weather'),
 (['How', "'s", 'the', 'weather', 'in', '[', 'city', ']', '?'], 'weather'),
 (['Tell', 'me', 'about', 'today', "'s", 'weather', 'forecast', '.'],
  'weather'),
 (['Will', 'it', 'be', 'sunny', 'this', 'weekend', '?'], 'weather'),
 (['Can', 'you', 'recommend', 'a', 'good', 'restaurant', '?'],
  'restaurant_recommendations'),
 (['I', "'m", 'looking', 'for', 'Italian', 'food', '.'],
  'restaurant_recommendations'),
 (['Any', 'sushi', 'restaurants', 'nearby', '?'],
  'restaurant_recommendations'),
 (['Where', 'can', 'I', 'find', 'a', 'vegetarian', 'restaurant', '?'],
  'restaurant_recommendations'),
 (['What', "'s", 'a', 'popular', 'place', 'for', 'brunch', '?'],
  'restaurant_recommendations'),
 

In [130]:
from nltk.corpus import stopwords
from nltk import punkt
import string

In [131]:
# now words has all the possible words in the corpus
# but now we want unique words therefore we use set 

unique_words = set(words)
# now we want the list of these unique words
words = list(unique_words)

words_final = []
for word in words:
    # if word not in stopwords.words('english') and word not in string.punctuation and word.isalnum():
    if word not in string.punctuation:
        words_final.append(word.lower())
# Adding all the words in lower case
words_final

['join',
 'support',
 'service',
 'get',
 'email',
 'positions',
 'name',
 'paypal',
 'my',
 'have',
 'tomorrow',
 'product',
 'much',
 'good',
 'good',
 'trouble',
 'when',
 'legal',
 'lawyer',
 'will',
 'arrive',
 'order',
 'change',
 'leave',
 'with',
 'hiring',
 'using',
 'payment',
 'rewards',
 'coverage',
 'consultation',
 'credit',
 'shipping',
 'insurance',
 'make',
 'located',
 'fee',
 'today',
 'book',
 'option',
 'about',
 "'s",
 'or',
 'flights',
 'there',
 'how',
 'can',
 'submit',
 'enroll',
 'morning',
 'find',
 'delete',
 'seminars',
 'promotions',
 'see',
 'received',
 'properly',
 'membership',
 'like',
 'looking',
 'update',
 'sunny',
 'going',
 'restaurants',
 'address',
 'having',
 'nearby',
 'place',
 'delivery',
 'products',
 'schedule',
 'question',
 'track',
 'after',
 'work',
 'your',
 'an',
 'international',
 'workshops',
 'where',
 'be',
 'feedback',
 'suggestion',
 'methods',
 'happening',
 'working',
 'account',
 'free',
 'problem',
 'status',
 'me',
 'tic

In [132]:
# words_final[] has all the unique words in lower case
words_final = [lemmatizer.lemmatize(word) for word in words_final]
# Now words_final has the lemmatized version of all the words
words_final

['join',
 'support',
 'service',
 'get',
 'email',
 'position',
 'name',
 'paypal',
 'my',
 'have',
 'tomorrow',
 'product',
 'much',
 'good',
 'good',
 'trouble',
 'when',
 'legal',
 'lawyer',
 'will',
 'arrive',
 'order',
 'change',
 'leave',
 'with',
 'hiring',
 'using',
 'payment',
 'reward',
 'coverage',
 'consultation',
 'credit',
 'shipping',
 'insurance',
 'make',
 'located',
 'fee',
 'today',
 'book',
 'option',
 'about',
 "'s",
 'or',
 'flight',
 'there',
 'how',
 'can',
 'submit',
 'enroll',
 'morning',
 'find',
 'delete',
 'seminar',
 'promotion',
 'see',
 'received',
 'properly',
 'membership',
 'like',
 'looking',
 'update',
 'sunny',
 'going',
 'restaurant',
 'address',
 'having',
 'nearby',
 'place',
 'delivery',
 'product',
 'schedule',
 'question',
 'track',
 'after',
 'work',
 'your',
 'an',
 'international',
 'workshop',
 'where',
 'be',
 'feedback',
 'suggestion',
 'method',
 'happening',
 'working',
 'account',
 'free',
 'problem',
 'status',
 'me',
 'ticket',
 'o

In [133]:
classes = sorted(list(set(classes)))
# There might be some words in the words_final that have the same lemmatized form 
# and we want all of the unique therefore we again find all the unique words
final_words = sorted(list(set(words_final)))

In [134]:
classes

['account_support',
 'appointment_booking',
 'booking_flights',
 'customer_support',
 'educational_courses',
 'event_information',
 'feedback',
 'general_info',
 'greeting',
 'insurance_information',
 'job_application',
 'legal_advice',
 'loyalty_program',
 'order_tracking',
 'payment_methods',
 'product_information',
 'restaurant_recommendations',
 'shipping_information',
 'tech_support',
 'weather']

In [135]:
final_words

["'m",
 "'s",
 'a',
 'about',
 'accept',
 'account',
 'additional',
 'address',
 'advice',
 'after',
 'an',
 'any',
 'apply',
 'appointment',
 'are',
 'arrive',
 'attend',
 'available',
 'be',
 'been',
 'benefit',
 'book',
 'booking',
 'brunch',
 'business',
 'can',
 'card',
 'cash',
 'certain',
 'certificate',
 'change',
 'charge',
 'choose',
 'city',
 'claim',
 'company',
 'complaint',
 'completing',
 'connect',
 'consultation',
 'contact',
 'cost',
 'course',
 'coverage',
 'credit',
 'customer',
 'date',
 'deductible',
 'delete',
 'delivery',
 'destination',
 'device',
 'do',
 'doctor',
 'doe',
 'earn',
 'educational',
 'email',
 'enroll',
 'event',
 'fee',
 'feedback',
 'find',
 'fix',
 'flight',
 'food',
 'for',
 'forecast',
 'forgot',
 'free',
 'get',
 'going',
 'good',
 'happening',
 'have',
 'having',
 'hello',
 'help',
 'hey',
 'hi',
 'hiring',
 'host',
 'hour',
 'how',
 'i',
 'improvement',
 'in',
 'information',
 'insurance',
 'international',
 'internship',
 'is',
 'issue',

In [136]:
pickle.dump(final_words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [137]:
#initializing training data
words = final_words
# just making life simpler so that i dont need to write final_words again and again
training = []
# here we are just assigning values 0 to all the output variables ie tags [0,0,0,0,0,0...]
output_empty = [0]*len(classes)
# bag is defined for the vectorized product
bag = []
for doc in documents:
    # this is the tokenized version of the elements of the question 
    token_ques = doc[0]
    # sice content of the document is not lemmatized hence we do lematizartion on it
    # we have just lemmatized the final_words
    token_word = [lemmatizer.lemmatize(word.lower()) for word in token_ques]
    for w in words:
        # now we have vectorized a particular question
        bag.append(1) if w in token_word else bag.append(0)
    # initialising the output list to 0
    output_row = list(output_empty)
    # output_row[doc[1]] = 1 this cannot be used as doc[1] is not a number
    # we want to put 1 at the index of the tag which is the tag of this particular question
    # and the rest are bydefault 0
    # to find the index of the tag we use classes.index(doc[1])
    output_row[classes.index(doc[1])] = 1
    # this traing consists of binary encoding of question and tag resp
    # training.append([bag,output_row])
    training.append([bag,output_row])
training



[[[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [138]:
from sklearn.model_selection import train_test_split

In [139]:
# training is a list and to do operations we need to convert it to a dataframe
training = pd.DataFrame(training)
training


,0,1
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...
95,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
96,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [140]:
X = training[0]
y = training[1]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [141]:
X

0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                            ...                        
95    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
96    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
97    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
98    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
99    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 0, Length: 100, dtype: object

In [142]:
y

0     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
1     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
2     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
3     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
4     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
                            ...                        
95    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
96    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
97    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
98    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
99    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 1, Length: 100, dtype: object

In [143]:
X_train_np = np.array(list(X_train))
y_train_np = np.array(list(y_train))



In [159]:
model = Sequential(
    [
        Dense(128, activation='relu', input_shape=(X_train_np.shape[1],)),
        Dropout(0.5),
        Dense(32,activation = 'relu'),
        Dropout(0.5),       
        Dense(y_train_np.shape[1],activation = 'softmax')
    ]
)


In [160]:
sgd = SGD(learning_rate=0.01, decay=1e-1, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# model.compile(optimizer='adam',
#     loss='categorical_crossentropy', metrics=['accuracy'])


In [161]:
history = model.fit(X_train_np, y_train_np, epochs=100, batch_size=5)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0413 - loss: 3.3344   
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0721 - loss: 2.9984 
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0693 - loss: 3.0516
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0715 - loss: 2.9861
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0565 - loss: 2.9899  
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0489 - loss: 2.9861 
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0674 - loss: 2.9783  
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0927 - loss: 2.9638 
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0873 - loss: 2.9849  
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0493 - loss: 2.9719 
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0307 - loss: 2.9724 
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/